## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D  # Convolution Operation
from keras.layers import MaxPooling2D # Pooling
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
from keras.layers import Activation
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [21]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

(50000, 32, 32, 3)


In [22]:
## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

In [23]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [29]:
num_classes = 10

model=Sequential()

#卷積組合
model.add(Conv2D(32, kernel_size=(3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(BatchNormalization(beta_initializer="zeros", gamma_initializer="ones")) 
model.add(Activation('relu'))
# max pooling & dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(BatchNormalization(beta_initializer="zeros", gamma_initializer="ones")) 
model.add(Activation('relu'))
# max pooling & dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128, kernel_size=(3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(BatchNormalization(beta_initializer="zeros", gamma_initializer="ones")) 
model.add(Activation('relu'))
# max pooling & dropout
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten
model.add(Flatten())

# FC
model.add(Dense(output_dim=num_classes,activation='softmax'))

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_31 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_29 (Batc (None, 16, 16, 64)      

C:\Users\Johnny\anaconda3\envs\Cupoy\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [30]:

#超過兩個就要選categorical_crossentrophy
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
50000/50000 [==============================] - 97s 2ms/step - loss: 1.5894 - accuracy: 0.4538
Epoch 2/10
50000/50000 [==============================] - 105s 2ms/step - loss: 1.1565 - accuracy: 0.5925
Epoch 3/10
50000/50000 [==============================] - 111s 2ms/step - loss: 1.0070 - accuracy: 0.6443
Epoch 4/10
50000/50000 [==============================] - 113s 2ms/step - loss: 0.9216 - accuracy: 0.6769
Epoch 5/10
50000/50000 [==============================] - 111s 2ms/step - loss: 0.8631 - accuracy: 0.6964
Epoch 6/10
50000/50000 [==============================] - 106s 2ms/step - loss: 0.8130 - accuracy: 0.7164
Epoch 7/10
50000/50000 [==============================] - 114s 2ms/step - loss: 0.7718 - accuracy: 0.7311
Epoch 8/10
50000/50000 [==============================] - 108s 2ms/step - loss: 0.7455 - accuracy: 0.7394
Epoch 9/10
50000/50000 [==============================] - 106s 2ms/step - loss: 0.7187 - accuracy: 0.7498
Epoch 10/10
50000/50000 [======================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [32]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
model.predict(input_example)

array([[6.2536083e-02, 1.4069562e-03, 1.5362082e-01, 4.3068427e-01,
        1.3245419e-01, 2.7561327e-02, 1.4678825e-01, 3.7826218e-02,
        6.9532879e-03, 1.6856493e-04]], dtype=float32)